In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from tqdm.notebook import tqdm


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
!pip install optuna

In [ ]:
import optuna

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/project9/train_transformed.csv')
train_labels = pd.read_csv('/content/drive/MyDrive/project9/music30s_trainlabel.csv')
test =pd.read_csv('/content/drive/MyDrive/project9/test_transformed.csv')


In [ ]:
train_labels =train_labels.iloc[:,1:]
train_labels.head(3)


,label
0,reggae
1,reggae
2,country


In [ ]:
cols = [c for c in train.columns]

In [ ]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
train[cols] = stdsc.fit_transform(train[cols])
test[cols] = stdsc.transform(test[cols])

In [ ]:
display(train.head(3))
display(test.head(3))

,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var
0,-0.298576,0.642278,0.177059,1.233038,1.386263,1.266091,0.764469,0.211645,0.314663,-0.234117,...,1.838597,0.833696,1.034494,0.776698,0.990850,1.481935,0.680471,0.666631,0.743034,-0.091657
1,-0.298576,0.227633,-0.595372,-0.050038,-0.362538,-0.174821,0.380594,-0.925636,-1.493513,0.877118,...,0.824710,-0.080086,0.785104,0.768889,1.027054,0.374397,0.127377,0.540703,-0.063098,0.060347
2,-0.142344,-0.050788,-0.477223,0.161218,0.571295,-0.000670,0.009428,0.213437,0.450461,-0.695707,...,0.631436,0.303991,-0.387986,0.346198,0.130285,-0.055250,-0.151197,-0.366009,-1.045659,-1.167858


,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var
0,-0.298576,-0.648987,-1.134700,-1.172775,-0.967173,-1.282262,-0.778708,-1.049366,-0.701822,0.365950,...,1.438900,1.531189,1.933873,1.593768,1.864126,0.803889,1.817925,1.818024,1.261632,1.923743
1,-0.142344,0.703835,-1.919805,-1.723546,-1.199076,-1.699029,-1.720368,0.207379,0.356319,0.877118,...,-1.153414,-1.881063,-3.674341,-3.609437,-3.750683,-3.791344,-3.700374,-3.402824,-2.418003,-1.934408
2,-0.298576,0.939176,2.349185,0.841052,0.373789,0.601497,-0.148723,0.212788,0.200619,-0.948482,...,1.184065,0.593270,1.220743,0.620820,0.712082,0.518463,0.468916,0.123271,-0.013371,0.258653


In [ ]:
train = pd.concat([train,train_labels],axis=1)

In [ ]:
train.head(3)

,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var,label
0,-0.298576,0.642278,0.177059,1.233038,1.386263,1.266091,0.764469,0.211645,0.314663,-0.234117,...,0.833696,1.034494,0.776698,0.990850,1.481935,0.680471,0.666631,0.743034,-0.091657,reggae
1,-0.298576,0.227633,-0.595372,-0.050038,-0.362538,-0.174821,0.380594,-0.925636,-1.493513,0.877118,...,-0.080086,0.785104,0.768889,1.027054,0.374397,0.127377,0.540703,-0.063098,0.060347,reggae
2,-0.142344,-0.050788,-0.477223,0.161218,0.571295,-0.000670,0.009428,0.213437,0.450461,-0.695707,...,0.303991,-0.387986,0.346198,0.130285,-0.055250,-0.151197,-0.366009,-1.045659,-1.167858,country


In [ ]:
N_SPLITS = 5
N_REPEATS = 3
EARLY_STOPPING_ROUNDS = 300
VERBOSE = False
SEED = 17171

In [ ]:
target_column = 'label'

In [ ]:
features = [c for c in train.columns if c not in [target_column]]

In [ ]:
X = train.drop('label',axis=1)
y=train['label']

In [ ]:
## going by the optuna way:
import optuna
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
# Which hyperparameters to tune: https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

def objective(trial):

    # -- Tune estimator algorithm
    n_neighbors = trial.suggest_int("n_neighbors", 1, 41,2)
    #suggest_int(name, low, high[, step, log])
    weights = trial.suggest_categorical("weights", ['uniform', 'distance'])
    metric = trial.suggest_categorical("metric", ['euclidean', 'manhattan', 'minkowski'])
    #algorithm{‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, default=’auto’
    algorithm = trial.suggest_categorical("algorithm",['auto', 'ball_tree', 'kd_tree', 'brute'])
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, metric=metric, algorithm =algorithm )
        
    # -- Cross-validate the features reduced by dimensionality reduction methods
    rskfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)
    score = cross_val_score(knn, X,y, scoring='accuracy', cv=rskfold, n_jobs = -1, error_score = 'raise')
    score = score.mean()
    return score


In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, timeout = 4*60*60)

[I 2022-04-24 09:34:13,790] A new study created in memory with name: no-name-1ad05584-a79e-4104-8837-8018250a78cf
[I 2022-04-24 09:34:15,762] Trial 0 finished with value: 0.6918518518518517 and parameters: {'n_neighbors': 7, 'weights': 'distance', 'metric': 'minkowski', 'algorithm': 'kd_tree'}. Best is trial 0 with value: 0.6918518518518517.
[I 2022-04-24 09:34:15,999] Trial 1 finished with value: 0.674074074074074 and parameters: {'n_neighbors': 1, 'weights': 'uniform', 'metric': 'minkowski', 'algorithm': 'brute'}. Best is trial 0 with value: 0.6918518518518517.
[I 2022-04-24 09:34:16,275] Trial 2 finished with value: 0.6914814814814815 and parameters: {'n_neighbors': 3, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'ball_tree'}. Best is trial 0 with value: 0.6918518518518517.
[I 2022-04-24 09:34:16,716] Trial 3 finished with value: 0.5814814814814816 and parameters: {'n_neighbors': 29, 'weights': 'uniform', 'metric': 'minkowski', 'algorithm': 'ball_tree'}. Best is trial 

In [ ]:
trial = study.best_trial    
print('Accuracy: {}'.format(trial.value))   


Accuracy: 0.7140740740740742


In [ ]:
print("Best hyperparameters: {}".format(trial.params))


Best hyperparameters: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'auto'}


In [ ]:

optuna.visualization.plot_optimization_history(study)


In [ ]:
knn_params = trial.params

In [ ]:
knn_params = {'algorithm': 'ball_tree',
 'metric': 'euclidean',
 'n_neighbors': 5,
 'weights': 'distance'}

In [ ]:

from sklearn.metrics import recall_score,precision_score,f1_score
from sklearn.metrics import classification_report
import statistics
from statistics import mean

In [116]:
%%time
N_SPLITS = 5
y_preds = []
acc= []
recall= []
f1= []
precision = []
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits = N_SPLITS)

for fold, (train_id, valid_id) in enumerate(folds.split(X,y)):
  X_train = X.iloc[train_id]
  y_train = y.iloc[train_id]
  X_valid = X.iloc[valid_id]
  y_valid = y.iloc[valid_id]
  model = KNeighborsClassifier(**knn_params)
  
  model.fit(X_train, y_train)
  valid_pred = model.predict(X_valid)

  #print('fold:{}'.format(fold+1))
  #print('Validation Accuracy score : {}'.format(accuracy_score(y_valid, valid_pred)))
  #print('-'*100)
  acc.append(accuracy_score(y_valid, valid_pred))
  recall.append(recall_score(y_valid, valid_pred, average = 'micro'))
  f1.append(f1_score(y_valid, valid_pred, average = 'micro'))
  precision.append(precision_score(y_valid, valid_pred, average = 'micro'))

  y_preds.append(model.predict(test))

print('the mean accuracy is: {}'.format(mean(acc)))

print('the mean precision is: {}'.format(mean(precision)))
print('the mean micro recall is: {}'.format(mean(recall)))

print('the mean micro f1_score is: {}'.format(mean(f1)))

the mean accuracy is: 0.6877777777777778
the mean precision is: 0.6877777777777778
the mean micro recall is: 0.6877777777777778
the mean micro f1_score is: 0.6877777777777778
CPU times: user 158 ms, sys: 0 ns, total: 158 ms
Wall time: 158 ms


In [117]:
y_preds

[array(['country', 'jazz', 'hiphop', 'reggae', 'country', 'reggae', 'pop',
        'classical', 'blues', 'blues', 'disco', 'disco', 'metal', 'reggae',
        'blues', 'country', 'country', 'reggae', 'country', 'blues',
        'jazz', 'hiphop', 'pop', 'classical', 'disco', 'disco', 'pop',
        'metal', 'country', 'jazz', 'jazz', 'classical', 'rock', 'jazz',
        'disco', 'country', 'classical', 'classical', 'pop', 'blues',
        'rock', 'blues', 'jazz', 'classical', 'hiphop', 'reggae', 'jazz',
        'blues', 'pop', 'pop', 'reggae', 'blues', 'hiphop', 'rock', 'jazz',
        'pop', 'blues', 'blues', 'reggae', 'pop', 'rock', 'pop', 'pop',
        'country', 'classical', 'rock', 'classical', 'reggae', 'hiphop',
        'metal', 'country', 'pop', 'metal', 'classical', 'classical',
        'country', 'disco', 'reggae', 'pop', 'country', 'disco', 'country',
        'hiphop', 'blues', 'classical', 'country', 'blues', 'disco',
        'disco', 'metal', 'country', 'blues', 'rock', 'm

In [ ]:
from collections import Counter
def mode(data):
    """Return the most common data point from discrete or nominal data.
    ``mode`` assumes discrete data, and returns a single value. This is the
    standard treatment of the mode as commonly taught in schools:
        >>> mode([1, 1, 2, 3, 3, 3, 3, 4])
        3
    This also works with nominal (non-numeric) data:
        >>> mode(["red", "blue", "blue", "red", "green", "red", "red"])
        'red'
    If there are multiple modes with same frequency, return the first one
    encountered:
        >>> mode(['red', 'red', 'green', 'blue', 'blue'])
        'red'
    If *data* is empty, ``mode``, raises StatisticsError.
    """
    pairs = Counter(iter(data)).most_common(1)
    try:
        return pairs[0][0]
    except IndexError:
        raise StatisticsError('no mode for empty data') from None




In [ ]:
y_pred = pd.DataFrame(columns=['label'])
pred= []
for j in range(0,100):
    arr = [y_preds[0][j], y_preds[1][j], y_preds[2][j], y_preds[3][j],y_preds[4][j]]
    pred.append(mode(arr))

In [ ]:
y_pred["label"]=pred

In [ ]:
y_pred

,label
0,country
1,jazz
2,hiphop
3,reggae
4,country
...,...
95,rock
96,disco
97,pop
98,pop


In [ ]:
y_pred.to_csv('knn_optuna_pred.csv', index=False)

In [ ]:
!cp -r '/content/knn_optuna_pred.csv' /content/drive/MyDrive/project9

